In [16]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('./train.csv')
df_test= pd.read_csv('./test.csv')

In [17]:
# GPU 가속
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.list_physical_devices('GPU') 이 코드도 가능
for gpu in gpus:
    print(gpu)

2024-07-29 20:04:33.248850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-29 20:04:33.248922: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [18]:
df_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# label 과 나머지 픽셀들을 분리
label = df_train['label'] # 픽셀들이 나타내는 값
data = df_train.copy()
data = data.drop(['label'], axis = 1)
label.info()
data.info()

<class 'pandas.core.series.Series'>
RangeIndex: 42000 entries, 0 to 41999
Series name: label
Non-Null Count  Dtype
--------------  -----
42000 non-null  int64
dtypes: int64(1)
memory usage: 328.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 251.2 MB


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(data, label, test_size=0.2, random_state=42)

In [6]:
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score

#radom forest
rmf = RandomForestClassifier()
rmf.fit(X_train, y_train)
Y_pred = rmf.predict(X_test)
rmf.score(X_test, y_test)
acc_rmf = round(rmf.score(X_test, y_test) * 100, 2)
acc_rmf

96.26

In [8]:
#xgboost
xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)
Y_pred = xgb.predict(X_test)
xgb.score(X_test, y_test)
acc_xgb = round(xgb.score(X_test, y_test) * 100, 2)
acc_xgb

97.42

In [9]:
#ensemble model -> randomforest, xgboost 모두 100퍼센트 나옴
from sklearn.ensemble import VotingClassifier

ves = VotingClassifier(estimators=[("Random Forest",rmf),
                                               ("XGBoost",xgb)],
                                  voting = 'soft')
ves.fit(X_train,y_train)
Y_pred_ves = ves.predict(X_test)
acc_ves = round(ves.score(X_test,y_test) * 100, 2)
acc_ves

97.37

## PFI - 하이퍼 파라미터 튜닝 전

In [23]:
# pfi - Xgboost
from sklearn.inspection import permutation_importance

pfi = permutation_importance(xgb, X_test, y_test,
                            n_repeats=30, random_state=0)

for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
    print(f"{X_test.columns[i] : <8}"
          f"{pfi.importances_mean[i] : .3f}"    
          f"+/- {pfi.importances_std[i] : .3f}")

pixel435 0.002+/-  0.000
pixel407 0.001+/-  0.000
pixel486 0.001+/-  0.000
pixel434 0.001+/-  0.000
pixel379 0.001+/-  0.000
pixel350 0.001+/-  0.000
pixel404 0.001+/-  0.000
pixel405 0.001+/-  0.000
pixel380 0.001+/-  0.000
pixel345 0.001+/-  0.000
pixel211 0.001+/-  0.000
pixel293 0.001+/-  0.000
pixel462 0.001+/-  0.000
pixel543 0.001+/-  0.000
pixel431 0.001+/-  0.000
pixel269 0.001+/-  0.000
pixel242 0.001+/-  0.000
pixel432 0.001+/-  0.000
pixel577 0.001+/-  0.000
pixel348 0.001+/-  0.000
pixel457 0.001+/-  0.000
pixel349 0.001+/-  0.000
pixel512 0.001+/-  0.000
pixel271 0.001+/-  0.000
pixel656 0.001+/-  0.000
pixel324 0.001+/-  0.000
pixel430 0.001+/-  0.000
pixel460 0.001+/-  0.000
pixel542 0.001+/-  0.000
pixel402 0.001+/-  0.000
pixel485 0.001+/-  0.000
pixel464 0.001+/-  0.000
pixel399 0.001+/-  0.000
pixel387 0.001+/-  0.000
pixel347 0.001+/-  0.000
pixel540 0.001+/-  0.000
pixel520 0.001+/-  0.000
pixel487 0.001+/-  0.000
pixel267 0.001+/-  0.000
pixel386 0.001+/-  0.000


In [24]:
# pfi - Random Forest
from sklearn.inspection import permutation_importance

pfi = permutation_importance(rmf, X_test, y_test,
                            n_repeats=30, random_state=0)

for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
    print(f"{X_test.columns[i] : <8}"
          f"{pfi.importances_mean[i] : .3f}"    
          f"+/- {pfi.importances_std[i] : .3f}")

pixel485 0.001+/-  0.000
pixel323 0.001+/-  0.000
pixel290 0.001+/-  0.000
pixel428 0.001+/-  0.000
pixel374 0.001+/-  0.000
pixel545 0.001+/-  0.000
pixel377 0.001+/-  0.000
pixel629 0.001+/-  0.000
pixel378 0.001+/-  0.000
pixel410 0.001+/-  0.000
pixel234 0.001+/-  0.000
pixel350 0.001+/-  0.001
pixel486 0.001+/-  0.000
pixel122 0.001+/-  0.000
pixel658 0.001+/-  0.000
pixel434 0.001+/-  0.000
pixel211 0.001+/-  0.000
pixel661 0.001+/-  0.000
pixel626 0.001+/-  0.000
pixel432 0.001+/-  0.000
pixel326 0.000+/-  0.000
pixel180 0.000+/-  0.000
pixel456 0.000+/-  0.000
pixel596 0.000+/-  0.000
pixel400 0.000+/-  0.000
pixel437 0.000+/-  0.000
pixel436 0.000+/-  0.000
pixel491 0.000+/-  0.000
pixel317 0.000+/-  0.000
pixel429 0.000+/-  0.000
pixel570 0.000+/-  0.000
pixel516 0.000+/-  0.000
pixel322 0.000+/-  0.000
pixel288 0.000+/-  0.000
pixel375 0.000+/-  0.000
pixel373 0.000+/-  0.000
pixel206 0.000+/-  0.000
pixel627 0.000+/-  0.000
pixel458 0.000+/-  0.000
pixel457 0.000+/-  0.000


In [26]:
# pfi - voting ensemble
from sklearn.inspection import permutation_importance

pfi = permutation_importance(ves, X_test, y_test,
                            n_repeats=30, random_state=0)

for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
    print(f"{X_test.columns[i] : <8}"
          f"{pfi.importances_mean[i] : .3f}"
          f"+/- {pfi.importances_std[i] : .3f}")

pixel435 0.001+/-  0.000
pixel211 0.001+/-  0.000
pixel350 0.001+/-  0.000
pixel486 0.001+/-  0.000
pixel293 0.001+/-  0.000
pixel345 0.001+/-  0.000
pixel401 0.001+/-  0.000
pixel269 0.001+/-  0.000
pixel399 0.001+/-  0.000
pixel380 0.001+/-  0.000
pixel379 0.001+/-  0.000
pixel407 0.001+/-  0.000
pixel457 0.001+/-  0.000
pixel349 0.001+/-  0.000
pixel485 0.001+/-  0.000
pixel404 0.001+/-  0.000
pixel343 0.001+/-  0.000
pixel292 0.001+/-  0.000
pixel319 0.001+/-  0.000
pixel323 0.001+/-  0.000
pixel434 0.001+/-  0.000
pixel387 0.001+/-  0.000
pixel577 0.001+/-  0.000
pixel267 0.001+/-  0.000
pixel400 0.001+/-  0.000
pixel571 0.001+/-  0.000
pixel243 0.001+/-  0.000
pixel270 0.001+/-  0.000
pixel370 0.001+/-  0.000
pixel205 0.001+/-  0.000
pixel268 0.001+/-  0.000
pixel298 0.001+/-  0.000
pixel461 0.001+/-  0.000
pixel441 0.001+/-  0.000
pixel440 0.001+/-  0.000
pixel405 0.001+/-  0.000
pixel317 0.001+/-  0.000
pixel371 0.001+/-  0.000
pixel431 0.001+/-  0.000
pixel378 0.001+/-  0.000


## 데이터 전처리 진행

In [7]:
X_train = X_train/255.0
X_test = X_test/255.0
df_test = df_test/255.0

In [8]:
X_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
5457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.model_selection import train_test_split

import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score

#radom forest
rmf = RandomForestClassifier()
rmf.fit(X_train, y_train)

#xgboost
xgb = xgboost.XGBClassifier(tree_method = 'gpu')
xgb.fit(X_train, y_train)

# voting ensemble
ves = VotingClassifier(estimators=[("Random Forest",rmf),
                                               ("XGBoost",xgb)],
                                  voting = 'soft')
ves.fit(X_train,y_train)

VotingClassifier(estimators=[('Random Forest', RandomForestClassifier()),
                             ('XGBoost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='multi:softprob',
                                            predictor=None, ...))],
                 voting='soft')

In [18]:
# pfi - Xgboost
from sklearn.inspection import permutation_importance

pfi = permutation_importance(xgb, X_test, y_test,
                            n_repeats=30, random_state=0)

for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
    print(f"{X_test.columns[i] : <8}"
          f"{pfi.importances_mean[i] : .3f}"    
          f"+/- {pfi.importances_std[i] : .3f}")

pixel435 0.002+/-  0.000
pixel407 0.001+/-  0.000
pixel486 0.001+/-  0.000
pixel434 0.001+/-  0.000
pixel379 0.001+/-  0.000
pixel350 0.001+/-  0.000
pixel404 0.001+/-  0.000
pixel405 0.001+/-  0.000
pixel380 0.001+/-  0.000
pixel345 0.001+/-  0.000
pixel211 0.001+/-  0.000
pixel293 0.001+/-  0.000
pixel462 0.001+/-  0.000
pixel543 0.001+/-  0.000
pixel431 0.001+/-  0.000
pixel269 0.001+/-  0.000
pixel242 0.001+/-  0.000
pixel432 0.001+/-  0.000
pixel577 0.001+/-  0.000
pixel348 0.001+/-  0.000
pixel457 0.001+/-  0.000
pixel349 0.001+/-  0.000
pixel512 0.001+/-  0.000
pixel271 0.001+/-  0.000
pixel656 0.001+/-  0.000
pixel324 0.001+/-  0.000
pixel430 0.001+/-  0.000
pixel460 0.001+/-  0.000
pixel542 0.001+/-  0.000
pixel402 0.001+/-  0.000
pixel485 0.001+/-  0.000
pixel464 0.001+/-  0.000
pixel399 0.001+/-  0.000
pixel387 0.001+/-  0.000
pixel347 0.001+/-  0.000
pixel540 0.001+/-  0.000
pixel520 0.001+/-  0.000
pixel487 0.001+/-  0.000
pixel267 0.001+/-  0.000
pixel386 0.001+/-  0.000


In [19]:
# pfi - Random Forest
from sklearn.inspection import permutation_importance

pfi = permutation_importance(rmf, X_test, y_test,
                            n_repeats=30, random_state=0)

for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
    print(f"{X_test.columns[i] : <8}"
          f"{pfi.importances_mean[i] : .3f}"    
          f"+/- {pfi.importances_std[i] : .3f}")

pixel350 0.001+/-  0.001
pixel405 0.001+/-  0.001
pixel153 0.001+/-  0.000
pixel151 0.001+/-  0.000
pixel513 0.000+/-  0.000
pixel318 0.000+/-  0.000
pixel461 0.000+/-  0.000
pixel637 0.000+/-  0.000
pixel437 0.000+/-  0.000
pixel372 0.000+/-  0.000
pixel288 0.000+/-  0.000
pixel155 0.000+/-  0.001
pixel485 0.000+/-  0.000
pixel636 0.000+/-  0.000
pixel343 0.000+/-  0.000
pixel688 0.000+/-  0.000
pixel177 0.000+/-  0.000
pixel148 0.000+/-  0.000
pixel383 0.000+/-  0.000
pixel377 0.000+/-  0.000
pixel181 0.000+/-  0.000
pixel208 0.000+/-  0.000
pixel147 0.000+/-  0.000
pixel605 0.000+/-  0.000
pixel101 0.000+/-  0.000
pixel329 0.000+/-  0.000
pixel438 0.000+/-  0.000
pixel606 0.000+/-  0.000
pixel260 0.000+/-  0.000
pixel638 0.000+/-  0.000
pixel202 0.000+/-  0.000
pixel553 0.000+/-  0.000
pixel123 0.000+/-  0.000
pixel455 0.000+/-  0.000
pixel581 0.000+/-  0.000
pixel568 0.000+/-  0.000
pixel124 0.000+/-  0.000
pixel631 0.000+/-  0.000
pixel400 0.000+/-  0.000
pixel175 0.000+/-  0.000


In [20]:
# pfi - Voting ensemble
from sklearn.inspection import permutation_importance

pfi = permutation_importance(ves, X_test, y_test,
                            n_repeats=30, random_state=0)

for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
    print(f"{X_test.columns[i] : <8}"
          f"{pfi.importances_mean[i] : .3f}"    
          f"+/- {pfi.importances_std[i] : .3f}")

pixel211 0.001+/-  0.000
pixel435 0.001+/-  0.000
pixel350 0.001+/-  0.000
pixel345 0.001+/-  0.000
pixel486 0.001+/-  0.000
pixel348 0.001+/-  0.000
pixel485 0.001+/-  0.000
pixel487 0.001+/-  0.000
pixel407 0.001+/-  0.000
pixel379 0.001+/-  0.000
pixel349 0.001+/-  0.000
pixel404 0.001+/-  0.000
pixel542 0.001+/-  0.000
pixel402 0.001+/-  0.000
pixel434 0.001+/-  0.000
pixel399 0.001+/-  0.000
pixel269 0.001+/-  0.000
pixel320 0.001+/-  0.000
pixel459 0.001+/-  0.000
pixel293 0.001+/-  0.000
pixel354 0.001+/-  0.000
pixel270 0.001+/-  0.000
pixel377 0.001+/-  0.000
pixel440 0.001+/-  0.000
pixel656 0.001+/-  0.000
pixel514 0.001+/-  0.000
pixel242 0.001+/-  0.000
pixel631 0.001+/-  0.000
pixel405 0.001+/-  0.000
pixel319 0.001+/-  0.000
pixel261 0.001+/-  0.000
pixel461 0.001+/-  0.000
pixel431 0.001+/-  0.000
pixel400 0.001+/-  0.000
pixel297 0.001+/-  0.000
pixel401 0.001+/-  0.000
pixel290 0.001+/-  0.000
pixel325 0.001+/-  0.000
pixel207 0.001+/-  0.000
pixel460 0.001+/-  0.000


In [11]:
sample = pd.read_csv('./sample_submission.csv')

In [13]:
ImageId = sample['ImageId']
ImageId

0            1
1            2
2            3
3            4
4            5
         ...  
27995    27996
27996    27997
27997    27998
27998    27999
27999    28000
Name: ImageId, Length: 28000, dtype: int64

In [14]:
Y_pred_rmf = rmf.predict(df_test)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_rmf
})
submission.to_csv('submission_rmf.csv', index = False)

Y_pred_xgb = xgb.predict(df_test)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_xgb
})
submission.to_csv('submission_xgb.csv', index = False)

Y_pred_ves = ves.predict(df_test)
submission = pd.DataFrame({
    "ImageId" : ImageId,
    "Label" : Y_pred_ves
})
submission.to_csv('submission_ves.csv', index = False)

/home/dummy/.local/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/dummy/.local/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/dummy/.local/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/dummy/.local/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtyp

In [9]:
data = data.to_numpy()
df_test = df_test.to_numpy()

data = data.reshape(-1,28,28)
df_test = df_test.reshape(-1,28,28)

In [10]:
X_train, X_test, y_train, y_test=train_test_split(data, label, test_size=0.2, random_state=42)

In [11]:
# CNN
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

def create_model():
    model = models.Sequential()
    # convolution layer
    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(28,28,1)))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), activation='relu'))

    # flatten layer
    model.add(layers.Flatten())

    # output layer
    model.add(layers.Dense(100, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'softmax'))

    model.summary()
    model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
             metrics = ['accuracy'])
    
    
    return model

In [12]:
model = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [ ]:
%%time
cnn = model.fit(X_train, y_train, epochs = 10,
                validation_data=(X_test, y_test))

Epoch 1/10


2024-07-29 19:53:05.070179: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 210739200 exceeds 10% of free system memory.
/home/dummy/anaconda3/envs/python/lib/python3.9/site-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1050/1050 [==============================] - ETA: 0s - loss: 0.3028 - accuracy: 0.9284

2024-07-29 19:53:14.323020: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 52684800 exceeds 10% of free system memory.


1050/1050 [==============================] - 9s 8ms/step - loss: 0.3028 - accuracy: 0.9284 - val_loss: 0.1056 - val_accuracy: 0.9638
Epoch 2/10
1050/1050 [==============================] - 8s 7ms/step - loss: 0.0790 - accuracy: 0.9754 - val_loss: 0.0660 - val_accuracy: 0.9794
Epoch 3/10
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0553 - accuracy: 0.9835 - val_loss: 0.0643 - val_accuracy: 0.9804
Epoch 4/10
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0473 - accuracy: 0.9851 - val_loss: 0.0624 - val_accuracy: 0.9813
Epoch 5/10
1050/1050 [==============================] - 8s 7ms/step - loss: 0.0396 - accuracy: 0.9883 - val_loss: 0.0647 - val_accuracy: 0.9824
Epoch 6/10
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0393 - accuracy: 0.9872 - val_loss: 0.0760 - val_accuracy: 0.9793
Epoch 7/10
1050/1050 [==============================] - 8s 7ms/step - loss: 0.0330 - accuracy: 0.9892 - val_loss: 0.0589 - val_accuracy: 0.9846
Epo

In [ ]:
cnn.save('./dev_cnn.h5')

# Optimization

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import RandomizedSearchCV
# 에러 해결을 위해 코드 추가
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'keras.wrappers'

In [21]:
def create_model_op(optimizer):
    model = models.Sequential()
    # convolution layer
    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(28,28,1)))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), activation='relu'))

    # flatten layer
    model.add(layers.Flatten())

    # output layer
    model.add(layers.Dense(100, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'softmax'))

    model.summary()
    model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
             metrics = ['accuracy'])
    
    
    return model

In [22]:
model = KerasClassifier(build_fn=create_model_op)
model

/tmp/ipykernel_576/2952964701.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model_op)


In [14]:
parameters = {'batch_size': [25, 32],
              'epochs': [30, 50],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy')

grid_search = grid_search.fit(X_train, y_train)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

/home/dummy/.local/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1076/1076 [==============================] - 5s 4ms/step - loss: 0.3431 - accuracy: 0.9253
Epoch 2/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0831 - accuracy: 0.9741
Epoch 3/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0588 - accuracy: 0.9825
Epoch 4/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0560 - accuracy: 0.9827
Epoch 5/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0422 - accuracy: 0.9868
Epoch 6/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0433 - accuracy: 0.9867
Epoch 7/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0376 - accuracy: 0.9882
Epoch 8/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0315 - accuracy: 0.9911
Epoch 9/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0289 - accuracy: 0.9916
Epoch 10/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0290 - accurac

/home/dummy/.local/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1076/1076 [==============================] - 5s 4ms/step - loss: 0.3444 - accuracy: 0.9209
Epoch 2/30
1076/1076 [==============================] - 4s 3ms/step - loss: 0.0838 - accuracy: 0.9747
Epoch 3/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0620 - accuracy: 0.9804
Epoch 4/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0518 - accuracy: 0.9841
Epoch 5/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0470 - accuracy: 0.9859
Epoch 6/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0380 - accuracy: 0.9881
Epoch 7/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0347 - accuracy: 0.9899
Epoch 8/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0299 - accuracy: 0.9912
Epoch 9/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0293 - accuracy: 0.9914
Epoch 10/30
1076/1076 [==============================] - 4s 4ms/step - loss: 0.0302 - accurac

...

840/840 [==============================] - 3s 3ms/step - loss: 0.4742 - accuracy: 0.9179
Epoch 2/50
840/840 [==============================] - 2s 3ms/step - loss: 0.0827 - accuracy: 0.9744
Epoch 3/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0576 - accuracy: 0.9814
Epoch 4/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0523 - accuracy: 0.9839
Epoch 5/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0448 - accuracy: 0.9865
Epoch 6/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0402 - accuracy: 0.9870
Epoch 7/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0362 - accuracy: 0.9881
Epoch 8/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0316 - accuracy: 0.9907
Epoch 9/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0295 - accuracy: 0.9916
Epoch 10/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0258 - accuracy: 0.9923
Epoch 11/5

/home/dummy/.local/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


840/840 [==============================] - 3s 3ms/step - loss: 0.3760 - accuracy: 0.9148
Epoch 2/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0872 - accuracy: 0.9735
Epoch 3/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0600 - accuracy: 0.9820
Epoch 4/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0503 - accuracy: 0.9841
Epoch 5/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0464 - accuracy: 0.9849
Epoch 6/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0386 - accuracy: 0.9877
Epoch 7/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0377 - accuracy: 0.9885
Epoch 8/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0346 - accuracy: 0.9900
Epoch 9/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0312 - accuracy: 0.9900
Epoch 10/50
840/840 [==============================] - 3s 3ms/step - loss: 0.0221 - accuracy: 0.9930
Epoch 11/5

/home/dummy/.local/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1050/1050 [==============================] - 4s 3ms/step - loss: 0.3486 - accuracy: 0.9263
Epoch 2/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0747 - accuracy: 0.9771
Epoch 3/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0565 - accuracy: 0.9817
Epoch 4/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0464 - accuracy: 0.9853
Epoch 5/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0404 - accuracy: 0.9882
Epoch 6/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0393 - accuracy: 0.9878
Epoch 7/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0324 - accuracy: 0.9898
Epoch 8/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0284 - accuracy: 0.9915
Epoch 9/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0271 - accuracy: 0.9918
Epoch 10/50
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0269 - accurac

In [18]:
# Retrieve the best model
best_model = grid_search.best_estimator_.model

# Evaluate performance on validation or test data
# loss, accuracy = best_model.evaluate(X_val, y_val)
# print("Validation accuracy:", accuracy)

# Save the best model
best_model.save("best_cnn_model.h5")

# Make predictions
predictions = best_model.predict(df_test)
predicted_labels = predictions.argmax(axis=1)  # Get the most probable digit for each image

# Create a submission DataFrame
submission_df = pd.DataFrame({"ImageId": range(1, len(df_test) + 1), "Label": predicted_labels})

# Save the submission file as a CSV
submission_df.to_csv("submission.csv", index=False)

print("Submission file created successfully!")
# kaggle score : 0.9856

875/875 [==============================] - 1s 1ms/step
Submission file created successfully!


In [27]:
from keras.models import load_model

model = load_model("best_cnn_model.h5")
pred = model.predict(X_test)
pred_label = pred.argmax(axis=1)
eval_result = model.evaluate(X_test, y_test)
print(f"Validation Accuracy: {eval_result[1]*100:.2f}%")

 70/263 [======>.......................] - ETA: 0s - loss: 0.2338 - accuracy: 0.9853 

/home/dummy/.local/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


263/263 [==============================] - 1s 2ms/step - loss: 0.2189 - accuracy: 0.9871
Validation Accuracy: 98.71%
